# CD1_P1

In [ ]:
modelname = 'CD1_P1'

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env into os.environ

platform = os.getenv("PLATFORM")

In [ ]:
# For Kaggle Notebooks ignore above cell & run this
# platform = 'Kaggle'

### Variable Paths, Execution Environments

In [ ]:
from utilpackage.Path import PathDefinition

# I use 3 Local Machines & 2 Cloud Compute Engines (Google, Kaggle). The appropriate paths for each platform of execution are declared here.
# Platform Names: 'Kaggle', 'Colab', 'Local', 'LocalRM', 'LocalOldLaptop'
pathBase, pathDataset, pathRawData, pathJoinedData, pathSavedModels = PathDefinition(platform)

### CD 1 Model, P1 Parameters

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16


### Load the Dataset

In [2]:
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
# Define paths to training and validation datasets
data_dir = pathDataset

# Load Datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='training',     # Use the 'training' subset
    seed=123
)

val_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='validation',   # Use the 'validation' subset
    seed=123
)

Found 24997 files belonging to 2 classes.
Using 19998 files for training.
Found 24997 files belonging to 2 classes.
Using 4999 files for validation.


### Preprocessing & Augmentation

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [5]:
# Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),  # 10% random rotation
    layers.RandomZoom(0.1),      # 10% zoom
    layers.RandomTranslation(0.1, 0.1),  # Random height and width shift
    layers.RandomBrightness(0.2)
])

# Augment the training data
def augment_img(image, label):
    image = data_augmentation(image)  # Apply augmentations
    return image, label

train_dataset = train_dataset.map(augment_img)

In [6]:
# Apply VGG-16 preprocessing
def preprocess_img(image, label):
    image = preprocess_input(image)  # Apply VGG16-specific preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess_img)
val_dataset = val_dataset.map(preprocess_img)

### Load & Freeze the Pre-trained Model

In [7]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
# Freeze the layers of VGG16 so we don't retrain them
for layer in vgg_base.layers:
    layer.trainable = False

### Add the Head in the New Model & Compile

In [9]:
model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Define the checkpoint path
model_name = model.name
checkpoint_path = f"{model_name}_head_epoch{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

### Train the Model

In [12]:
history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9234 - loss: 2.9528
Epoch 1: saving model to sequential_1_head_epoch001_val0.0849.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3502s 6s/step - accuracy: 0.9234 - loss: 2.9499 - val_accuracy: 0.9816 - val_loss: 0.0849
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9583 - loss: 0.1269
Epoch 2: saving model to sequential_1_head_epoch002_val0.0556.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3356s 5s/step - accuracy: 0.9583 - loss: 0.1269 - val_accuracy: 0.9842 - val_loss: 0.0556


### Save History in JSON

In [ ]:
import json
import numpy as np

model_name = model.name
# Assuming 'history' is the keras.callbacks.History object
filepath = f"{model_name}_2epochs_training_history.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)

### Continue Training

In [14]:
# Define the checkpoint path
model_name = 'CD_1P1_2_6epochs'
checkpoint_path = f"{model_name}_head_epoch{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

In [15]:
history2 = model.fit(
    train_dataset,
    epochs=4,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9591 - loss: 0.1118
Epoch 1: saving model to CD_1P1_2_6epochs_head_epoch001_val0.0660.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3035s 5s/step - accuracy: 0.9591 - loss: 0.1118 - val_accuracy: 0.9830 - val_loss: 0.0660
Epoch 2/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9633 - loss: 0.0991
Epoch 2: saving model to CD_1P1_2_6epochs_head_epoch002_val0.0605.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 2990s 5s/step - accuracy: 0.9633 - loss: 0.0991 - val_accuracy: 0.9846 - val_loss: 0.0605
Epoch 3/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9630 - loss: 0.1029
Epoch 3: saving model to CD_1P1_2_6epochs_head_epoch003_val0.0664.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 4405s 7s/step - accuracy: 0.9630 - loss: 0.1029 - val_accuracy: 0.9826 - val_loss: 0.0664
Epoch 4/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9634 - loss: 0.1100
Epoch 4: saving model to CD_1P1_2_6epochs_head_epoch004_val0.0686.keras
625/625 ━━━━━━━━━━━

In [ ]:
import json
import numpy as np

model_name = model.name
# Assuming 'history' is the keras.callbacks.History object
filepath = f"{model_name}_2epochs_training_history.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)